<a href="https://colab.research.google.com/github/sergeymasl/pandas_cource/blob/main/Data_Types_and_Missing_Values.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Data Types and Missing Values**

## **Введение**

В этом уроке вы изучете какие типы данных есть в DataFrame и Series.
Также вы узнаете как заменять данные в DataFrame и Series.

Для демонстрации мы будем использовать датасет с данными о винах.

In [ ]:
# импорт библиотеки
import pandas as pd
# загрузка датасета для дальнейшей работы
reviews = pd.read_csv("https://drive.google.com/uc?export=download&id=1z-1idT4mGbOvHgmEPzneqV54EJ1-w7tk", index_col=0)
# настройки по отображению строк
pd.set_option('max_rows', 5)

## **Типы данных**

>**Типы данных** -  это очень важная вещь, на которую **стоит потратить время** в самом начале вашей работы, то есть при загрузки данных.

>В противном случае, вы можете столкнуться с неправильными результатами или ошибками слишком поздно и на исправление этих ошибок уйдет много-много больше времени и нервов, чем вы бы потратили на работу с типами данных при загрузке.

Есть некоторая разница между типами данных в Pandas, NumPy и Python. Вот сравнительная табличка:

|Pandas        | Python        | NumPy                                                         | Описание                                         |
|---           |---            |---                                                            |---                                                    |
|object        |str или смесь  |string_, unicode_, смешанные типы                              | Текстовые или смешанные числовые и нечисловые значения|
|int64         |int            |int_, int8, int16, int32, int64, uint8, uint16, uint32, uint64 | Целые числа                                           |
|float64       |float          |float_, float16, float32, float64                              | Числа с плавающей точкой                              |
|bool          |bool           |bool_                                                          | Значения True/False                                   |
|datetime64    |datetime             |datetime64[ns]                                                 | Значения даты и времени                               |
|timedelta[ns] |NA             |NA                                                             | Разность между двумя datetimes                        |
|category      |NA             |NA                                                             | Ограниченный список текстовых значений                |

In [ ]:
reviews['price'].dtype

dtype('float64')

Еще один метод, позволяющий узнать типы данных это ```dtypes```. Он возвращяет тип данных для каждого столбца в DataFrame:

In [ ]:
reviews.dtypes

country        object
description    object
                ...  
variety        object
winery         object
Length: 13, dtype: object

Тип данных говорит нам немного о том как Pandas хранит внутри себя данные.
>```float64``` означает что это числа с плавающей запятой размером 64 бита (или 8 байт)

>```int64``` означает целое число размером 64 бита (или 8 байт)

Особенность, которую необходимо запомнить, столбцы со строковым типом данных получают тип данных ```object```.